In [1]:
import os
import random
import time
from typing import Tuple

import numpy as np
import torch
import tqdm
from PIL import Image
import cv2 as cv
import numpy
import math
import utils
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer
from nuscenes.eval.common.utils import quaternion_yaw
from pyquaternion import Quaternion
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from nuscenes.prediction.input_representation.static_layers import correct_yaw
from matplotlib import pyplot as plt
from nuscenes.map_expansion.bitmap import BitMap
import threading
from threading import Thread, Lock
import multiprocessing as mp
from utils import NuscenesUtil

In [2]:
nusc = NuScenes(version='v1.0-trainval', dataroot='E:\\nuScenesDataset\\v1.0-trainval_meta', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 37.645 seconds.
Reverse indexing ...
Done reverse indexing in 8.3 seconds.


In [3]:
nusc_map = NuScenesMap(map_name='singapore-hollandvillage', dataroot='E:\\nuScenesDataset\\v1.0-trainval_meta')

map_locs = ['singapore-onenorth', 'singapore-hollandvillage', 'singapore-queenstown', 'boston-seaport']
maps = {item: NuScenesMap(map_name=item, dataroot='E:\\nuScenesDataset\\v1.0-trainval_meta') for item in map_locs}


helper = PredictHelper(nusc)
# static_layer_rasterizer = StaticLayerRasterizer(helper, meters_ahead=80, meters_left=50, meters_right=50)
static_layer_rasterizer = StaticLayerRasterizer(helper)
# agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1, meters_ahead=80, meters_left=50, meters_right=50)
agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1)
mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())
bitmap = BitMap(nusc_map.dataroot, nusc_map.map_name, 'basemap')

In [4]:
challenges = get_prediction_challenge_split('train', 'E:\\nuScenesDataset\\v1.0-trainval_meta')
target_instance_token, sample_token = challenges[900].split('_')
# target_instance_token, sample_token = challenges[200].split('_')
target_ann = helper.get_sample_annotation(target_instance_token, sample_token)

In [5]:
def sort_by_distance(item):
    return item['distance']

def get_surrounding_vehicle_by_distance(sample_token, target_instance_token, top_k):
    """
    get all vehicles in a specific sample
    :param sample_token:
    :return: [(annotation_token, instance_token), ...]
    """
    target_ann = helper.get_sample_annotation(target_instance_token, sample_token)
    target_x, target_y = target_ann['translation'][0], target_ann['translation'][1]

    # get vehicles with specific category
    annotations = helper.get_annotations_for_sample(sample_token)
    category_filter = NuscenesUtil.get_type_list()
    result = [ann for ann in annotations if ann['category_name'] in category_filter]

    # sort them by the distance to target vehicle and filte top 15
    for item in result:
        distance = NuscenesUtil.point_distance(item['translation'][0], item['translation'][1], target_x, target_y)
        item['distance'] = distance

    result.sort(key=sort_by_distance)
    result = result[:top_k]
    result = [item['token'] for item in result]

    return result

def get_vehicles_ann_token_in_past(ann_token, time_horizon):
    """
    get annotation token of vehicles in the past
    :param ann_token: annotation token of target vehicle
    :param time_horizon:
    :return:
    """
    ann = nusc.get('sample_annotation', ann_token)
    instance_token, sample_token = ann['instance_token'], ann['sample_token']
    past_ann = helper.get_past_for_agent(instance_token, sample_token, time_horizon, False, False)
    past_ann_token = [ann['token'] for ann in past_ann]
    return past_ann_token


In [6]:
# node feature
def generate_node_feature(get_ann_by_index, target_instance_token, target_sample_token):
    node_features = []
    node_num = len(get_ann_by_index)
    target_ann = helper.get_sample_annotation(target_instance_token, target_sample_token)
    target_x, target_y, target_yaw = target_ann['translation'][0], target_ann['translation'][1], NuscenesUtil.get_correct_yaw(target_ann['rotation'])
    # the positions of all of vehicles is relative to the position of target vehicle in the last timestep
    for node_id in range(node_num):
        ann_token = get_ann_by_index[node_id]
        ann = nusc.get('sample_annotation', ann_token)
        # position
        x, y, yaw = ann['translation'][0], ann['translation'][1], NuscenesUtil.get_correct_yaw(ann['rotation'])
        if ann_token != target_ann['token']:
            x, y, yaw = NuscenesUtil.align_coordinate(target_x, target_y, target_yaw, x, y, yaw)
        else:
            x, y, yaw = 0, 0, 0
        # velocity, acceleration, type
        velocity = helper.get_velocity_for_agent(ann['instance_token'], ann['sample_token'])
        acceleration = helper.get_acceleration_for_agent(ann['instance_token'], ann['sample_token'])
        type = NuscenesUtil.type_with_one_hot_encoding(ann['category_name'])
        # construct feature by position, velocity, acceleration and type
        feature = [x, y, yaw, velocity, acceleration]
        feature.extend(type)
        node_features.append(feature)

    node_features = torch.tensor(node_features)
    node_features = torch.nan_to_num(node_features)
    return node_features

# edge feature
def generate_edge_feature(edges, node_features):
    edge_features = torch.zeros(len(edges), 5)
    for index, edge in enumerate(edges):
        from_node, to_node = edge
        feature = node_features[from_node][:5] - node_features[to_node][:5]
        edge_features[index][:] = feature
    return edge_features

def generate_vehicle_graph(challenge):
    target_instance_token, target_sample_token = challenge.split('_')
    target_ann = helper.get_sample_annotation(target_instance_token, target_sample_token)
    # generate graph
    # node
    # get all vehicle in past sample
    past = helper.get_past_for_agent(target_instance_token, target_sample_token, 5, False, False)
    past_sample_token = [target_sample_token]
    past_sample_token.extend([item['sample_token'] for item in past])
    past_vehicles = [get_surrounding_vehicle_by_distance(sample_token, target_instance_token, 20) for sample_token in past_sample_token]

    # generate node index where node is a specific vehicle in a specific sample
    get_index_by_ann = {}
    get_ann_by_index = {}
    index = 0
    for vehicles in past_vehicles:
        for vehicle in vehicles:
            get_index_by_ann[vehicle] = index
            get_ann_by_index[index] = vehicle
            index += 1

    # edge
    edges = []

    # for the same timestep, connect surrounding vehicles with 5 directions to the target vehicle
    for vehicles in past_vehicles:
        for vehicle in vehicles:
            ann = nusc.get('sample_annotation', vehicle)
            instance_token, sample_token = ann['instance_token'], ann['sample_token']
            surrounding_vehicles = NuscenesUtil.get_surrounding_vehicle(helper, nusc_map, instance_token, sample_token)
            for key in surrounding_vehicles:
                surrounding_vehicle = surrounding_vehicles[key]
                surrounding_ann_token = surrounding_vehicle['token']
                if surrounding_ann_token not in get_index_by_ann:
                    continue
                from_index, to_index = get_index_by_ann[surrounding_ann_token], get_index_by_ann[vehicle]
                edges.append((from_index, to_index))

    # for the vehicle in a specific timestep, connect it to itself and surrounding vehicles in the past 1 seconds
    for vehicles in past_vehicles:
        for vehicle in vehicles:
            # get past annotation_token
            past_ann_tokens = get_vehicles_ann_token_in_past(vehicle, 1)
            # connect to itself and surrounding vehicles
            for past_ann_token in past_ann_tokens:
                if past_ann_token not in get_index_by_ann:
                    continue
                ann = nusc.get('sample_annotation', past_ann_token)
                instance_token, sample_token = ann['instance_token'], ann['sample_token']
                edges.append((get_index_by_ann[past_ann_token], get_index_by_ann[vehicle]))
                surrounding_vehicles = NuscenesUtil.get_surrounding_vehicle(helper, nusc_map, instance_token, sample_token)
                for key in surrounding_vehicles:
                    surrounding_vehicle = surrounding_vehicles[key]
                    surrounding_ann_token = surrounding_vehicle['token']
                    if surrounding_ann_token not in get_index_by_ann:
                        continue
                    from_index, to_index = get_index_by_ann[surrounding_ann_token], get_index_by_ann[vehicle]
                    edges.append((from_index, to_index))

    node_features = generate_node_feature(get_ann_by_index, target_instance_token, target_sample_token)
    edge_features = generate_edge_feature(edges, node_features)

    return node_features, edge_features, edges, get_index_by_ann[target_ann['token']]


In [21]:
node_features, edge_features, edges, target_node_index = generate_vehicle_graph(challenges[3100])
node_features.shape, edge_features.shape, len(edges), target_node_index

(torch.Size([165, 14]), torch.Size([1061, 5]), 1061, 0)

In [22]:
result = {}
for challenge in tqdm.tqdm(challenges):
    node_features, edge_features, edges, target_node_index = generate_vehicle_graph(challenge)
    result[challenge] = (node_features, edge_features, edges, target_node_index)

100%|██████████| 32186/32186 [3:40:10<00:00,  2.44it/s]   


In [23]:
torch.save(result, './preprocess_vehicle_graph/train.data')

In [24]:
challenges = get_prediction_challenge_split('val', 'E:\\nuScenesDataset\\v1.0-trainval_meta')
result = {}
for challenge in tqdm.tqdm(challenges):
    node_features, edge_features, edges, target_node_index = generate_vehicle_graph(challenge)
    result[challenge] = (node_features, edge_features, edges, target_node_index)
torch.save(result, './preprocess_vehicle_graph/test.data')

100%|██████████| 9041/9041 [1:20:29<00:00,  1.87it/s]  


In [7]:
train_data = torch.load('./preprocess_vehicle_graph/train.data')

In [8]:
challenge = challenges[0]
train_data[challenge]

(tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  4.8726e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 3.2686e+00,  1.3640e+00,  1.5527e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 3.4878e+00,  6.9412e+00,  1.6399e+00,  2.1266e-02,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.0878e+01, -3.3116e+00,  4.7058e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.1160e+01,  3.2221e+00,  4.7528e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00

In [47]:
import random

for _ in range(10):
    challenge = random.choice(challenges)
    node_features, edge_features, edges, tar = train_data[challenge]
    num_node, num_edge = len(node_features), len(edges)
    print(f'{num_node}, {num_edge}, {num_edge / num_node:.2f}')

120, 191, 1.59
165, 1183, 7.17
131, 213, 1.63
105, 589, 5.61
42, 42, 1.00
165, 252, 1.53
165, 260, 1.58
4, 2, 0.50
165, 345, 2.09
74, 108, 1.46
